In [7]:
import os
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import sys
import tensorflow as tf
import time
from pylab import *
import numpy as np
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
from skimage.util.shape import view_as_windows
from sklearn.metrics import roc_curve,roc_auc_score
from tcn import TCN, tcn_full_summary
from tensorflow.keras.layers import Dense, Conv2D, Layer, Lambda, Flatten, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, Model, metrics
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import plot_model
from tensorflow.python.client import device_lib

import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

module_path = os.path.abspath(os.path.join('..'))
project_path = os.path.abspath(os.path.join('../..'))
train_source = os.path.abspath(os.path.join('../../data/train_source/'))
test_source = os.path.abspath(os.path.join('../../data/test_source/'))
train_path = os.path.abspath(os.path.join('../../data/train/'))
test_path = os.path.abspath(os.path.join('../../data/test/'))
dest_path_bsu = os.path.abspath(os.path.join('../../data/final_bsu/'))
dest_path_bs = os.path.abspath(os.path.join('../../data/final_bs/'))
val_path_bs = os.path.abspath(os.path.join('../../data/validation_bs/'))
val_path_bsu = os.path.abspath(os.path.join('../../data/validation_bsu/'))

if module_path not in sys.path:
    sys.path.append(module_path)
if project_path not in sys.path:
    sys.path.append(project_path)
    
np.set_printoptions(suppress=True)
np.set_printoptions(threshold=sys.maxsize)
#print(device_lib.list_local_devices())

Using TensorFlow backend.


In [4]:
# Hyperparameters
nb_classes=2
cols, rows = 2, 30
input_shape = (cols, rows, 1)
vol_imb_diff = 0.1
evaluate_every = 100 # interval for evaluating on one-shot tasks
batch_size = 32
n_iter = 20000 # No. of training iterations
n_val = 250 # how many one-shot tasks to validate on
h = 30
w = 2
d = 2

In [5]:
num_frames = 10
# FOR BS Training 
# Train
X_train = np.load(dest_path_bs + '/' + str(num_frames) + '_X_train.npy')
Y_train = np.load(dest_path_bs + '/' + str(num_frames) + '_Y_train.npy')
F_train = np.load(dest_path_bs + '/' + str(num_frames) + '_F_train.npy')
X_test = np.load(dest_path_bs + '/' + str(num_frames) + '_X_test.npy')
Y_test = np.load(dest_path_bs + '/' + str(num_frames) + '_Y_test.npy')
F_test = np.load(dest_path_bs + '/' + str(num_frames) + '_F_test.npy')

print(X_train.shape)
print(X_test.shape)

#X_train = np.append(X_train, X_test, axis=0)
#Y_train = np.append(Y_train, Y_test, axis=0)
#F_train = np.append(F_train, F_test, axis=0)
#print(X_train.shape)

# Validation
X_val = np.load(val_path_bs + '/' + str(num_frames) + '_X_val.npy')
Y_val = np.load(val_path_bs + '/' + str(num_frames) + '_Y_val.npy')
print(X_val.shape)

(265825, 10, 30, 2, 2)
(93741, 10, 30, 2, 2)
(19733, 10, 30, 2, 2)


In [20]:
class CategoricalTruePositives(tf.keras.metrics.Metric):
    def __init__(self, num_classes, batch_size,
                 name="categorical_true_positives", **kwargs):
        super(CategoricalTruePositives, self).__init__(name=name, **kwargs)

        self.batch_size = batch_size
        self.num_classes = num_classes    

        self.cat_true_positives = self.add_weight(name="ctp", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):     

        y_true = K.argmax(y_true, axis=-1)
        y_pred = K.argmax(y_pred, axis=-1)
        y_true = K.flatten(y_true)

        true_poss = K.sum(K.cast((K.equal(y_true, y_pred)), dtype=tf.float32))
        self.cat_true_positives.assign_add(true_poss)

    def result(self):
        return self.cat_true_positives

In [39]:
# create and fit the LSTM network
labels_train = K.one_hot(Y_train, nb_classes+1)
X_train_r = X_train.reshape((X_train.shape[0], X_train.shape[1], 120))

model = Sequential()
model.add(LSTM(4, input_shape=(10, 120)))

#output_dim is a categorical variable with 8 classes
model.add(Dense(activation='softmax', units=3))

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print(model.summary())
#model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_r, labels_train, epochs=200, batch_size=None, steps_per_epoch=10, verbose=1, sample_weight=F_train, validation_split=0.02)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 4)                 2000      
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 15        
Total params: 2,015
Trainable params: 2,015
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
10/10 [==============================] - 35s 4s/step - loss: 0.2923 - accuracy: 0.1742
Epoch 2/200
10/10 [==============================] - 36s 4s/step - loss: 0.2832 - accuracy: 0.2205
Epoch 3/200
10/10 [==============================] - 36s 4s/step - loss: 0.2763 - accuracy: 0.2762
Epoch 4/200
10/10 [==============================] - 35s 3s/step - loss: 0.2713 - accuracy: 0.3464
Epoch 5/200
10/10 [==============================] - 30s 3s/step - loss: 0.2675 - accuracy: 0.4322
Epoch 6/200
1

In [40]:
X_val_r = X_val.reshape((X_val.shape[0], X_val.shape[1], 120))
pred = model.predict(X_val_r)

In [41]:
print(pred[0])
labels_val = K.one_hot(Y_val, nb_classes+1)
r = tf.keras.metrics.Recall()
r.update_state(labels_val, pred)
r.result().numpy()
print('Recall ' + str(r.result()))

p = tf.keras.metrics.Precision()
p.update_state(labels_val, pred)
p.result().numpy()
print('Precision ' + str(p.result()))

f1 = 2*((p.result()*r.result())/(p.result()+r.result()+K.epsilon()))
print('F1 ' + str(f1))


[0.10338772 0.6311512  0.26546106]
Recall tf.Tensor(0.49328536, shape=(), dtype=float32)
Precision tf.Tensor(0.7043925, shape=(), dtype=float32)
F1 tf.Tensor(0.5802336, shape=(), dtype=float32)


In [43]:
hist = model.evaluate(X_val_r, labels_val)
# summarize history for accuracy
plt.plot(hist['accuracy'])
plt.plot(hist['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(hist['loss'])
plt.plot(hist['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

ValueError: If your data is in the form of symbolic tensors, you should specify the `steps` argument (instead of the `batch_size` argument, because symbolic tensors are expected to produce batches of input data).